### Importar librerías

In [ ]:
import numpy as np
import pandas as pd
import os
import datetime
import psutil
from sklearn import preprocessing
import matplotlib.pyplot as plt
from scipy.stats import shapiro

from commons import myfunctions as myfunc

start_time = datetime.datetime.now()


### Variables globales


In [ ]:
myfunc.NOTEBK_FILENAME = myfunc.get_nb_name()

myfunc.reset_vars()


### Representación de los valores de metilación en algunas muestras

Lee ficheros de metilación por genes

In [ ]:
df_t = myfunc.read_h5_to_df("rows_transpose_by_gene_id_with_target_num", myfunc.H5_DIR)


Escoger zonas donde la varianza sea mayor

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

#  coger una muestra de cada tipo
muestras=pd.DataFrame()

for tipo1 in set(df_t.target):
    muestra1 = df_t[df_t.target == tipo1].sample(1, random_state=42)
    if muestras.empty:
        muestras = muestra1.copy()
    else:
        muestras = pd.concat([muestras, muestra1], axis = 0)

X = muestras.iloc[:, :-1]
y = muestras.iloc[:, -1]

# Seleccionar las características
selector = SelectKBest(f_classif, k=100)
X_selected = selector.fit_transform(X, y)

mask_columnas_seleccionadas = selector.get_support()
columnas_seleccionadas = X.columns[mask_columnas_seleccionadas]

df1 = pd.merge(X[columnas_seleccionadas], y, left_index=True, right_index=True)


Representación gráfica del perfil utilizando las muestras seleccionadas anteriormente

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

muestras = df1.copy()

max_value = muestras.max().max()

fig, axes = plt.subplots(muestras.shape[0], 1, figsize=(14,10))

for i in range(len(muestras)):
    axes[i].bar(range(muestras.shape[1]), muestras.iloc[i])
    axes[i].set_ylabel(f"Tipo {int(muestras.iloc[i]['target'])}")
    axes[i].set_ylim([0, max_value])
    axes[i].set_xticks([])

# solo se muestran las etiquetas del eje x al final
axes[-1].set_xticks(range(0, muestras.shape[1], 10))
axes[-1].set_xlabel("Genes")

plt.show()


Ver la normalidad de los datos utilizando la prueba de Shapiro-Wilk

In [ ]:
plt.hist(df1)
plt.show()

shapiro_result = shapiro(df1)
p_value = shapiro_result.pvalue

if p_value > 0.05:
    print(f"Los datos parecen seguir una distribución normal {p_value}")
else:
    print(f"Los datos NO parecen seguir una distribución normal {p_value}")


### Explorar datos de las muestras

Lee ficheros con todas las muestras

In [ ]:
df_t = myfunc.read_h5_to_df("files_joined", myfunc.H5_DIR)

columns_to_drop = ["Chr", "Start", "End", "Strand", "Length"]
df_t.drop(columns_to_drop, axis=1, inplace=True)

df_t.set_index('id', inplace=True)

df_t = df_t.transpose()

display(df_t)


Leer fichero de clases

In [ ]:
df_c = myfunc.read_h5_to_df("classes_by_id", myfunc.H5_DIR)

display(df_c)


Algunos estadísticos sobre las muestras

In [ ]:
myfunc.verbose("Some statistics by rows")

df_calc = df_t.copy()

df_calc['CntCols']       = df_t.count(axis=1)
df_calc['CntZeros']      =(df_t == 0).sum(axis=1)
df_calc['NonZeros']      =(df_t != 0).sum(axis=1)
df_calc['CntNaN']        = df_t.isnull().sum(axis=1)
df_calc['SumCols']       = df_t.sum(axis=1)
df_calc['MaxValue']      = df_t.max(axis=1)
df_calc['MinValue']      = df_t.min(axis=1)
df_calc['VarValue']      = df_t.var(axis=1)
df_calc['StdValue']      = df_t.std(axis=1)
df_calc['MeanValue']     = df_t.mean(axis=1)
df_calc['DistinctValues']= df_t.nunique(axis=1)

display(df_calc)

if myfunc.SHOW_DISPLAYS:
    display(df_calc.shape)
    df_calc.info()


In [ ]:
display(df_calc.sort_values("MaxValue"))

#### Ver la distribución de algunas muestras en el stream

### Para contar los distintos valores aplanamos el df

In [ ]:
import matplotlib.pyplot as plt
flattened_data = df_t.values.flatten()
print(flattened_data.shape)


In [ ]:
from collections import Counter
display(flattened_data.shape)

Conteo = Counter(flattened_data)

top_10 = Conteo.most_common(10)
total_elementos = len(flattened_data)

porcentajes = [(numero, count, (count / total_elementos) * 100) for numero, count in top_10]

for numero, count, porcentaje in porcentajes:
    print(f"Número: {numero}, Cantidad: {count}, Porcentaje: {porcentaje:.2f}%")

Ver el tramo entre 0 y 100

In [ ]:
vmin=0
vmax=100
filtered_data = flattened_data[(flattened_data >= vmin) & (flattened_data <= vmax)]
display(filtered_data.shape)

plt.figure(figsize=(20,5))
plt.hist(filtered_data, bins=range(vmin,vmax), edgecolor='black', alpha=0.7)
plt.title('Histograma de Valores en el DataFrame')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()


In [ ]:
vmin=100
vmax=400
filtered_data = flattened_data[(flattened_data >= vmin) & (flattened_data <= vmax)]
display(filtered_data.shape)

plt.figure(figsize=(20,5))
plt.hist(filtered_data, bins=range(vmin,vmax), edgecolor='black', alpha=0.7)
plt.title('Histograma de Valores en el DataFrame')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()


In [ ]:
vmin=400
vmax=1800
filtered_data = flattened_data[(flattened_data >= vmin) & (flattened_data <= vmax)]
display(filtered_data.shape)

plt.figure(figsize=(20,5))
plt.hist(filtered_data, bins=range(vmin,vmax), edgecolor='black', alpha=0.7)
plt.title('Histograma de Valores en el DataFrame')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()


### veamos el resultado quitando la muestra Z82

In [ ]:
import matplotlib.pyplot as plt
if "Z8" in df_t.columns:
    flattened_data = df_t.drop('Z82').values.flatten()
print(flattened_data.shape)


In [ ]:
filtered_data = flattened_data[(flattened_data >= 56) & (flattened_data <= 2000)]
display(filtered_data.shape)

print(filtered_data.shape[0]/flattened_data.shape[0],'%')

plt.figure(figsize=(20,5))
plt.hist(filtered_data, bins=range(450,1005), edgecolor='black', alpha=0.7)
plt.title('Histograma de Valores en el DataFrame')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()


#### Finalización del notebook

In [ ]:
end_time = datetime.datetime.now()
total_time = end_time - start_time
myfunc.verbose(f"Notebook ha tardado {total_time} segundos")
